In [10]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv('labeledTrainData.csv',
                   header=0 , delimiter = '\t',
                   quoting=3)

In [3]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
train.shape

(25000, 3)

In [7]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [20]:
print(train['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [17]:
example = BeautifulSoup(train['review'][0])

I:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file I:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [21]:
print(example.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [29]:
letters_only = re.sub('[^a-zA-Z]',' ',example.get_text())
#Find anything that is NOT a lowercase letter (a-z) or an upper case letter (A-Z), and replace it with a space.
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [25]:
#lower and split called 'tokenization'

In [30]:
lower_case = letters_only.lower()
words = lower_case.split()

In [6]:
#去掉一些额外的词-  of a that for the等
#这种词称为 停止词 stop words
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [7]:
from nltk.corpus import stopwords

In [39]:
print(stopwords.words('english'))
#英语的stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [41]:
words = [w for w in words if not w in stopwords.words('english')]

In [42]:
words #去掉评论中的stop words

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [8]:
def review_to_words(raw_review):
    bs = BeautifulSoup(raw_review)
    
    words = re.sub('[^a-zA-Z]' , ' ' , bs.get_text())
    words = set(words.lower().split())
    #set 搜索速度比list快
    
    words = [w for w in words if not w in stopwords.words('english')]
    
    return ' '.join(words)

In [57]:
clean_review = review_to_words(train['review'][0])

I:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file I:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [58]:
clean_review

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [9]:
num_reviews = train['review'].size
clean_train_reviews = []

for i in range(num_reviews):
    if (i+1)%1000 == 0:
        print('processing %d' % (i))
    
    clean_train_reviews.append(review_to_words(train['review'][i]))

I:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file I:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


processing 999
processing 1999
processing 2999
processing 3999
processing 4999
processing 5999
processing 6999
processing 7999
processing 8999
processing 9999
processing 10999
processing 11999
processing 12999
processing 13999
processing 14999
processing 15999
processing 16999
processing 17999
processing 18999
processing 19999
processing 20999
processing 21999
processing 22999
processing 23999
processing 24999


In [12]:
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None , 
                            preprocessor = None , 
                            stop_words = None ,
                            max_features = 5000) #出现频率最高的前5000个单词
train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()
#得到bag of words

In [13]:
train_data_features.shape #查看转换后的review的shape
#25000 5000

(25000, 5000)

In [14]:
vocab = vectorizer.get_feature_names()
print(vocab)
#看看是哪5000个单词 是最频繁的

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abrupt', 'absence', 'absent', 'absolute', 'absolutely', 'absorbed', 'absurd', 'absurdity', 'abuse', 'abused', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepts', 'access', 'accident', 'accidentally', 'acclaimed', 'accompanied', 'accomplish', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'active', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'addicted', 'adding', 'addition', 'additional', 'address', 'adds', 'adequate', 'admirable', 'admire', 'admit', 'admittedly', 'adolescent', 'adopted', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair'

In [18]:
len(vocab)

5000

In [19]:
#extra
dist = np.sum(train_data_features , axis = 0)
#纵向相加 就是每个单词在所有评论张的出现次数总和 （一共5000个频繁单词）
for tag , count in zip(vocab , dist):
    print(count , tag)
#上面是输出 单词 与 频数

177 abandoned
82 abc
101 abilities
415 ability
1143 able
73 abrupt
110 absence
81 absent
334 absolute
1383 absolutely
67 absorbed
281 absurd
68 absurdity
172 abuse
71 abused
84 abusive
96 abysmal
257 academy
430 accent
184 accents
271 accept
123 acceptable
70 acceptance
135 accepted
70 accepts
87 access
290 accident
192 accidentally
75 acclaimed
88 accompanied
72 accomplish
121 accomplished
286 according
171 account
80 accuracy
263 accurate
114 accused
167 achieve
132 achieved
119 achievement
75 acid
892 across
1070 act
638 acted
5361 acting
2307 action
282 actions
73 active
75 activities
1973 actor
3620 actors
1054 actress
335 actresses
373 acts
707 actual
3441 actually
138 ad
190 adam
73 adams
364 adaptation
73 adaptations
148 adapted
779 add
420 added
71 addicted
164 adding
334 addition
73 additional
70 address
319 adds
111 adequate
69 admirable
115 admire
588 admit
133 admittedly
71 adolescent
68 adopted
100 adorable
439 adult
330 adults
99 advance
84 advanced
147 advantage
426 adv

1944 ending
97 endings
223 endless
888 ends
97 endure
94 enemies
171 enemy
296 energy
93 engage
106 engaged
296 engaging
273 england
789 english
1619 enjoy
786 enjoyable
1140 enjoyed
160 enjoying
144 enjoyment
109 enjoys
101 enormous
2875 enough
140 ensemble
87 ensues
181 enter
126 enters
169 entertain
226 entertained
1341 entertaining
803 entertainment
80 enthusiasm
1291 entire
495 entirely
68 entitled
136 entry
157 environment
272 epic
985 episode
642 episodes
136 equal
412 equally
82 equipment
86 equivalent
72 er
538 era
190 eric
172 erotic
96 errors
461 escape
102 escaped
153 escapes
2268 especially
133 essence
153 essential
250 essentially
156 established
108 estate
78 et
1022 etc
67 eternal
70 ethan
184 europe
254 european
87 eva
102 eve
8415 even
216 evening
324 event
768 events
663 eventually
4804 ever
3300 every
383 everybody
156 everyday
1932 everyone
2031 everything
188 everywhere
189 evidence
125 evident
70 evidently
1072 evil
406 ex
176 exact
919 exactly
119 exaggerated
72

141 julie
273 jump
90 jumped
120 jumping
151 jumps
71 june
130 jungle
79 junior
179 junk
347 justice
96 justify
99 juvenile
111 kane
72 karen
219 kate
138 keaton
1479 keep
266 keeping
601 keeps
68 keith
185 kelly
92 ken
80 kennedy
99 kenneth
696 kept
231 kevin
387 key
236 kick
87 kicked
89 kicking
124 kicks
924 kid
103 kidding
122 kidnapped
1268 kids
1020 kill
916 killed
932 killer
190 killers
614 killing
118 killings
447 kills
160 kim
2299 kind
257 kinda
187 kinds
613 king
79 kingdom
67 kirk
154 kiss
77 kissing
111 kitchen
822 knew
127 knife
68 knight
124 knock
73 knocked
4781 know
431 knowing
256 knowledge
989 known
810 knows
176 kong
86 korean
71 kubrick
126 kudos
131 kung
110 kurt
391 la
96 lab
954 lack
121 lacked
269 lacking
77 lackluster
341 lacks
263 ladies
633 lady
155 laid
178 lake
646 lame
316 land
71 landing
74 lands
102 landscape
74 landscapes
120 lane
482 language
520 large
215 largely
138 larger
136 larry
2470 last
92 lasted
1097 late
92 lately
1836 later
192 latest
70 la

184 politics
143 pool
1599 poor
628 poorly
289 pop
104 popcorn
71 popping
85 pops
504 popular
80 popularity
103 population
277 porn
83 porno
96 portion
132 portrait
254 portray
463 portrayal
70 portrayals
552 portrayed
216 portraying
220 portrays
168 position
475 positive
72 positively
102 possessed
94 possibilities
103 possibility
929 possible
665 possibly
437 post
114 poster
73 posters
94 pot
567 potential
88 potentially
119 poverty
101 powell
750 power
542 powerful
264 powers
225 practically
97 practice
164 praise
70 praised
283 pre
118 precious
74 precisely
774 predictable
174 prefer
153 pregnant
655 premise
75 prepare
169 prepared
73 preposterous
386 presence
556 present
150 presentation
380 presented
72 presenting
191 presents
192 president
113 press
77 pressure
121 presumably
110 pretend
92 pretending
244 pretentious
2913 pretty
121 prevent
85 preview
579 previous
198 previously
72 prey
243 price
81 priceless
124 pride
166 priest
102 primarily
99 primary
189 prime
75 primitive
1

284 woods
128 woody
838 word
814 words
95 wore
3515 work
595 worked
116 worker
136 workers
746 working
1148 works
2883 world
124 worlds
85 worn
112 worried
140 worry
1271 worse
2269 worst
2089 worth
117 worthless
181 worthwhile
348 worthy
8131 would
89 wound
77 wounded
391 wow
85 wrap
88 wrapped
115 wreck
77 wrestling
74 wretched
624 write
1007 writer
595 writers
88 writes
1159 writing
1462 written
1592 wrong
533 wrote
70 ww
136 wwii
91 ya
74 yard
427 yeah
1945 year
3647 years
82 yelling
87 yellow
1319 yes
113 yesterday
2271 yet
618 york
2718 young
452 younger
72 youngest
248 youth
319 zero
329 zombie
257 zombies
125 zone


In [22]:
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(train_data_features , train['sentiment'])

In [23]:
#ring test
test = pd.read_csv('testData.csv' , header=0,
                  delimiter = '\t',
                  quoting = 3)
test.shape

(25000, 2)

In [24]:
num_reviews = len(test['review'])
clean_test_reviews = []

for i in range(num_reviews):
    if (i+1)%1000 == 0:
        print('processing %d' % (i))
    
    clean_test_reviews.append(review_to_words(test['review'][i]))

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

I:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file I:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


processing 999
processing 1999
processing 2999
processing 3999
processing 4999
processing 5999
processing 6999
processing 7999
processing 8999
processing 9999
processing 10999
processing 11999
processing 12999
processing 13999
processing 14999
processing 15999
processing 16999
processing 17999
processing 18999
processing 19999
processing 20999
processing 21999
processing 22999
processing 23999
processing 24999


In [25]:
output = pd.DataFrame(data={'id':test['id'] , 'sentiment':result})

output.to_csv('submit.csv' , index = False , quoting = 3)

In [26]:
np.log(8)

2.0794415416798357

In [28]:
np.log(1.0/(1+np.exp(-0.8)))

-0.37110066594777763

In [29]:
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

sigmoid(0)

0.5

In [39]:
logits = sigmoid(sigmoid(np.array([0.2,0.4,0.7,0.9])))

In [40]:
label = np.array([0,0,1,1])

In [41]:
np.multiply(label , np.log(logits)) + np.multiply(1-label , np.log(1-logits))

array([-1.00538722, -1.03664083, -0.41385432, -0.39956614])